<a href="https://colab.research.google.com/github/jinensetpal/panoptic-reproducibility/blob/main/encoder_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl https://rclone.org/install.sh | sudo bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4491  100  4491    0     0   6941      0 --:--:-- --:--:-- --:--:--  6930
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    15  100    15    0     0     16      0 --:--:-- --:--:-- --:--:--    16
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.8M  100 13.8M    0     0  7788k      0  0:00:01  0:00:01 --:--:-- 7784k
Archive:  rclone-current-linux-amd64.zip
   creating: tmp_unzip_dir_for_rclone/rclone-v1.55.1-linux-amd64/
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.55.1-linux-amd64/README.txt  [text]  
  inflating: tmp_unzip_dir_for_rclone/rclone-v1.55.1-linux-amd64/rclone.1  [text]  
  inflating:

In [ ]:
# !rclone config ; cp /root/.config/rclone/rclone.conf .
!wget https://66931f1295a8.ngrok.io/rclone.conf ; mkdir -p /root/.config/rclone/ ; cp rclone.conf /root/.config/rclone/

--2021-06-27 06:36:47--  https://66931f1295a8.ngrok.io/rclone.conf
Resolving 66931f1295a8.ngrok.io (66931f1295a8.ngrok.io)... 3.134.125.175, 2600:1f16:d83:1200::6e:3
Connecting to 66931f1295a8.ngrok.io (66931f1295a8.ngrok.io)|3.134.125.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3686 (3.6K) [text/plain]
Saving to: ‘rclone.conf’

rclone.conf         100%[===================>]   3.60K  --.-KB/s    in 0s      

2021-06-27 06:36:47 (59.1 MB/s) - ‘rclone.conf’ saved [3686/3686]



In [ ]:
!sudo mkdir /content/onedrive
!nohup rclone --vfs-cache-mode writes mount colab: /content/onedrive &

In [ ]:
import tensorflow as tf
from keras.applications.xception import Xception

In [ ]:
encoder = Xception(input_shape=(1029, 2049, 3),
                      include_top=False,
                      weights='imagenet')

encoder.summary()

83697664/83683744 [==============================] - 1s 0us/step
Model: "xception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1029, 2049,  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 514, 1024, 32 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 514, 1024, 32 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 514, 1024, 32 0           block1_conv1_bn[0][0]            
__________________________

In [ ]:
!pip install tf_slim

     |████████████████████████████████| 358kB 5.1MB/s 


In [ ]:
conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)
conv3 = Dropout(0.2)(conv3)
conv3 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv3)

up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
conv4 = Dropout(0.2)(conv4)
conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)

up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
conv5 = Dropout(0.2)(conv5)
conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)

In [ ]:
import tensorflow as tf
from keras.applications.Xception import Xception

latent_dim = 64
class EncoderDecoder(Xception):
  def __init__(self, latent_dim):
    super(EncoderDecoder, self).__init__()
    self.latent_dim = latent_dim   
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(latent_dim, activation='relu'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(784, activation='sigmoid'),
      layers.Reshape((28, 28))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = Autoencoder(latent_dim)


In [ ]:
!pip install keras_segmentation tf_slim

In [ ]:
import tf_slim as slim

@slim.add_arg_scope
def atrous_spatial_pyramid_pooling(net, scope, depth=256):
  with tf.variable_scope(scope):
    feature_map_size = tf.shape(net)

    # apply global average pooling
    image_level_features = tf.reduce_mean(net, [1, 2], name='image_level_global_pool', keep_dims=True)
    image_level_features = slim.conv2d(image_level_features, depth, [1, 1], scope="image_level_conv_1x1", activation_fn=None)
    image_level_features = tf.image.resize_bilinear(image_level_features, (feature_map_size[1], feature_map_size[2]))

    at_pool1x1 = slim.conv2d(net, depth, [1, 1], scope="conv_1x1_0", activation_fn=None)

    at_pool3x3_1 = slim.conv2d(net, depth, [3, 3], scope="conv_3x3_1", rate=6, activation_fn=None)

    at_pool3x3_2 = slim.conv2d(net, depth, [3, 3], scope="conv_3x3_2", rate=12, activation_fn=None)

    at_pool3x3_3 = slim.conv2d(net, depth, [3, 3], scope="conv_3x3_3", rate=18, activation_fn=None)

    net = tf.concat((image_level_features, at_pool1x1, at_pool3x3_1, at_pool3x3_2, at_pool3x3_3), axis=3, name="concat")
    net = slim.conv2d(net, depth, [1, 1], scope="conv_1x1_output", activation_fn=None)
    return net

In [ ]:
from keras.applications.xception import Xception

def deeplab_v3(inputs, args, is_training, reuse):

    # mean subtraction normalization
    inputs = inputs - [_R_MEAN, _G_MEAN, _B_MEAN]

    # inputs has shape [batch, 513, 513, 3]
    with slim.arg_scope(resnet_utils.resnet_arg_scope(args.l2_regularizer, is_training,
                                                      args.batch_norm_decay,
                                                      args.batch_norm_epsilon)):
        xception = Xception(input_shape=(1029, 2049, 3),
                          include_top=False,
                          weights='imagenet')
        _, end_points = xception(inputs,
                                 args.number_of_classes,
                                 is_training=is_training,
                                 global_pool=True,
                                 spatial_squeeze=False,
                                 output_stride=args.output_stride,
                                 reuse=reuse)

        with tf.variable_scope("DeepLab_v3", reuse=reuse):

            # get block 4 feature outputs
            net = end_points[args.resnet_model + '/block4']
            net = atrous_spatial_pyramid_pooling(net, "ASPP_layer", depth=256, reuse=reuse)
            net = slim.conv2d(net, args.number_of_classes, [1, 1], activation_fn=None,
                              normalizer_fn=None, scope='logits')

            size = tf.shape(inputs)[1:3]
            net = tf.image.resize_bilinear(net, size)
            return net

In [17]:
import tensorflow as tf
from keras import Model
from keras.applications.xception import Xception
from tensorflow.keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Add
from keras_segmentation.models.model_utils import get_segmentation_model

# encoder-decoder
def buildmodel(input_img, n_classes=17, TRAINABLE=False):

    base_model = Xception(input_shape=(1029, 2049, 3),
                          include_top=False,
                          weights='imagenet')

    for layer in base_model.layers:
        layer.trainable=TRAINABLE
    
#-------------------encoder---------------------------- 
#--------(pretrained & trainable if selected)----------

    x=base_model.get_layer('block1_conv1')(input_img)
    x=base_model.get_layer('block1_conv1_bn')(x)
    x=base_model.get_layer('block1_conv1_act')(x)
    x=base_model.get_layer('block1_conv2')(x)
    x=base_model.get_layer('block1_conv2_bn')(x)
    x=base_model.get_layer('block1_conv2_act')(x)

#    block1 -- 1/4
    x=base_model.get_layer('block2_sepconv1')(x)
    # x=base_model.get_layer('block2_sepconv1')(input_img)
    x=base_model.get_layer('block2_sepconv1_bn')(x)
    x=base_model.get_layer('block2_sepconv2_act')(x)
    x=base_model.get_layer('block2_sepconv2')(x)
    x=base_model.get_layer('block2_sepconv2_bn')(x)
    x=base_model.get_layer('conv2d')(x)
    # x=Conv2D(256, (1, 1), activation=None, padding='same')(x)
    x=base_model.get_layer('block2_pool')(x)
    x=base_model.get_layer('batch_normalization')(x)
    # x=BatchNormalization(axis=-1)(x)
    ups1=base_model.get_layer('add')(x)

#    block2 -- 1/8
    x=base_model.get_layer('block3_sepconv1_act')(ups1)
    x=base_model.get_layer('block3_sepconv1')(x)
    x=base_model.get_layer('block3_sepconv1_bn')(x)
    x=base_model.get_layer('block3_sepconv2_act')(x)
    x=base_model.get_layer('block3_sepconv2')(x)
    x=base_model.get_layer('block3_sepconv2_bn')(x)
    x=base_model.get_layer('conv2d_1')(x)
    x=base_model.get_layer('block3_pool')(x)
    x=base_model.get_layer('batch_normalization_1')(x)
    ups2=base_model.get_layer('add_1')(x)

#    block3 -- 1/16
    x=base_model.get_layer('block4_sepconv1_act')(ups2)
    x=base_model.get_layer('block4_sepconv1')(x)
    x=base_model.get_layer('block4_sepconv1_bn')(x)    
    x=base_model.get_layer('block4_sepconv2_act')(x)
    x=base_model.get_layer('block4_sepconv2')(x)
    x=base_model.get_layer('block4_sepconv2_bn')(x)
    x=base_model.get_layer('conv2d_2')(x)
    x=base_model.get_layer('block4_pool')(x)
    x=base_model.get_layer('batch_normalization_2')(x)
    x=base_model.get_layer('add_2')(x)

#    block4 -- 1/16
    x=base_model.get_layer('block5_sepconv1_act')(x)
    x=base_model.get_layer('block5_sepconv1')(x)
    x=base_model.get_layer('block5_sepconv1_bn')(x)    
    x=base_model.get_layer('block5_sepconv2_act')(x)
    x=base_model.get_layer('block5_sepconv2')(x)
    x=base_model.get_layer('block5_sepconv2_bn')(x)
    x=base_model.get_layer('block5_sepconv3_act')(x)
    x=base_model.get_layer('block5_sepconv3')(x)
    x=base_model.get_layer('block5_sepconv3_bn')(x)
    x=base_model.get_layer('add_3')(x)
    
#--------latent space (trainable) ------------
    x = atrous_spatial_pyramid_pooling(x, scope="aspp")
#--------------decoder (trainable)----------- 
    # Block 5    
    x = Conv2D(256, (1, 1), activation='relu', padding='same')(x)
    x = UpSampling2D((2,2))(x)

    # Block 6 
    x = Conv2D(256, (1, 1), activation='relu', padding='same')(x)
    x = concatenate([UpSampling2D((2, 2))(x), ups1], axis=-1)

    # Block 7
    x = Conv2D(256, (1, 1), activation='relu', padding='same')(x)
    x = UpSampling2D((2,2))(x)
    x = concatenate([UpSampling2D((2, 2))(x), ups2], axis=-1)

    # Block 8
    x = Conv2D(256, (5, 5), activation='relu', padding='same')(x)

    # Block 9 
    x = Conv2D(256, (5, 5), activation='relu', padding='same')(x)

    # Block 10 
    x = Conv2D(n_classes, (5, 5), activation='relu', padding='same')(x)
    
    return get_segmentation_model(input_img, x) 

input_image = Input(shape = (1029, 2049, 3))

seg = buildmodel(input_image)
seg.compile(loss='mean_squared_error', optimizer = Adam())
seg.summary()

The following Variables were used a Lambda layer's call (tf.nn.convolution), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1/kernel:0' shape=(3, 3, 3, 32) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.compat.v1.nn.fused_batch_norm_16), but
are not present in its tracked objects:
  <tf.Variable 'block1_conv1_bn/gamma:0' shape=(32,) dtype=float32>
  <tf.Variable 'block1_conv1_bn/beta:0' shape=(32,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (tf.nn.convolution_1), but
are not present in its tracked objects:
  <tf.Variable

ValueError: ignored